In [3]:
import os
from google.cloud import bigquery
import seaborn as sns
import pandas as pd
import numpy as np
import shutil

# Authenticate With Bigquery:

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = \
'D:/GoogleDrive/GoogleCloudServiceAccountsKey/BigQueryServiceAccountForJupyter/api-7223922200765231820-28682-14f1efc35761.json'

In [6]:
query = """
SELECT
    date,
    airline,
    airline_code,
    departure_airport,
    departure_state,
    departure_lat,
    departure_lon,
    arrival_airport,
    arrival_state,
    arrival_lat,
    arrival_lon,
    departure_schedule,
    departure_actual,
    departure_delay,
    arrival_schedule,
    arrival_actual,
    arrival_delay
FROM
  `bigquery-samples.airline_ontime_data.flights`
LIMIT 10
"""

In [7]:
airlineDF = bigquery.Client().query(query).to_dataframe()

In [9]:
airlineDF.head()

,date,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,arrival_airport,arrival_state,arrival_lat,arrival_lon,departure_schedule,departure_actual,departure_delay,arrival_schedule,arrival_actual,arrival_delay
0,2007-10-30,OO,20304,ATL,GA,33.63,-84.42,HOU,TX,29.64,-95.27,941,936,-5.0,1050,1043,-7.0
1,2009-02-27,OO,20304,ATL,GA,33.63,-84.42,HOU,TX,29.64,-95.27,1050,1103,13.0,1210,1233,23.0
2,2009-02-23,OO,20304,ATL,GA,33.63,-84.42,HOU,TX,29.64,-95.27,1050,1047,-3.0,1210,1207,-3.0
3,2009-02-24,OO,20304,ATL,GA,33.63,-84.42,HOU,TX,29.64,-95.27,1050,1057,7.0,1210,1234,24.0
4,2007-10-08,OO,20304,ATL,GA,33.63,-84.42,HOU,TX,29.64,-95.27,941,1029,48.0,1050,1242,112.0


<h3> Create a simple machine learning model </h3>

The dataset that we will use is <a href="https://bigquery.cloud.google.com/table/bigquery-samples:airline_ontime_data.flights">a BigQuery public dataset</a> of airline arrival data. Click on the link, and look at the column names. Switch to the Details tab to verify that the number of records is 70 million, and then switch to the Preview tab to look at a few rows.
<p>
We want to predict the arrival delay of an airline based on the departure delay. The model that we will use is a zero-bias linear model:
$$ delay_{arrival} = \alpha * delay_{departure} $$
<p>
To train the model is to estimate a good value for $\alpha$. 
<p>
One approach to estimate alpha is to use this formula:
$$ \alpha = \frac{\sum delay_{departure} delay_{arrival} }{  \sum delay_{departure}^2 } $$
Because we'd like to capture the idea that this relationship is different for flights from New York to Los Angeles vs. flights from Austin to Indianapolis (shorter flight, less busy airports), we'd compute a different $alpha$ for each airport-pair.  For simplicity, we'll do this model only for flights between Denver and Los Angeles.

<h2> Naive random split (not repeatable) </h2>

In [10]:
compute_alpha = """
SELECT
    SAFE_DIVIDE(SUM(departure_delay * arrival_delay), SUM(departure_delay * departure_delay)) AS alpha
FROM
(
SELECT 
    RAND() AS splitField,
    arrival_delay,
    departure_delay
FROM
    `bigquery-samples.airline_ontime_data.flights`
WHERE
    departure_airport = 'DEN' AND arrival_airport='LAX'
)
WHERE
    splitField < 0.8
"""

In [11]:
resultsDF = bigquery.Client().query(compute_alpha).to_dataframe()

In [12]:
resultsDF.head()

,alpha
0,0.974889


In [13]:
alpha = resultsDF['alpha'][0]
print(alpha)

0.9748890625806536
